In [1]:
import numpy as np
import pandas as pd

##### 1. Implement the Naïve Bayes classifier using user define functions.

In [2]:
data = {"Color":['Red', 'Red', 'Blue', 'Blue', 'Yellow', 'Yellow', 'Red'],
        "Type":['SUV', 'XUV', 'SUV', np.nan, 'XUV', 'SUV', 'SUV'],
        "Mileage":[13.2, 12.1, 14.8, 14.5, 12.2, np.nan, 11.8],
        "Origin":['Imported', 'Domestic', 'Domestic', 'Imported', 'Domestic', 'Imported', 'Imported'],
        "Stolen":['Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No']
       }
df = pd.DataFrame(data)
df

,Color,Type,Mileage,Origin,Stolen
0,Red,SUV,13.2,Imported,Yes
1,Red,XUV,12.1,Domestic,Yes
2,Blue,SUV,14.8,Domestic,No
3,Blue,NaN,14.5,Imported,No
4,Yellow,XUV,12.2,Domestic,Yes
5,Yellow,SUV,NaN,Imported,Yes
6,Red,SUV,11.8,Imported,No


In [3]:
#replacing null values in Type with most frequent Type
Type_mode = df['Type'].value_counts().idxmax()
df['Type'].replace(np.nan, Type_mode, inplace=True)
Type_mode

'SUV'

In [4]:
#replacing empty Mileage with mean
Mileage_mean = df['Mileage'].mean()
df['Mileage'].replace(np.nan, Mileage_mean, inplace=True)
Mileage_mean

13.1

In [5]:
df

,Color,Type,Mileage,Origin,Stolen
0,Red,SUV,13.2,Imported,Yes
1,Red,XUV,12.1,Domestic,Yes
2,Blue,SUV,14.8,Domestic,No
3,Blue,SUV,14.5,Imported,No
4,Yellow,XUV,12.2,Domestic,Yes
5,Yellow,SUV,13.1,Imported,Yes
6,Red,SUV,11.8,Imported,No


In [6]:
"""
Class naiveBayes is a user build function to Naive Bayes Classifier.
User have to pass Dataframe with target column name.
User can predict the class using predict fuction and have to pass inputs in a dict.
"""
class naiveBayes():
    
    def __init__(self, df, target):
        self.data = df
        self.prob_dict = {}
        self.target = target
        self.clss_lst = df[self.target].unique().tolist()
        self.prob_dict['total'] = df[self.target].count()
        for _ in self.clss_lst:
            self.prob_dict[f"total_{_}"] = df[df[self.target] == _][self.target].count()
            self.prob_dict[f"prob_{_}"] = round(self.prob_dict[f"total_{_}"] / self.prob_dict['total'], 4)
    
    def get_cond_prob(self, feature, values):
        sdf = df[df[feature] == values]
        laplacian_flag = False
        for _ in self.clss_lst:
            self.prob_dict[f"total_{values}_{_}"] = sdf[sdf[self.target] == _][self.target].count()
            E = self.prob_dict[f"total_{values}_{_}"]
            S = self.prob_dict[f"total_{_}"]
            
            # Below condition to handle laplacian.
            if self.prob_dict[f"total_{values}_{_}"] == 0 or laplacian_flag == True:
                print("Encounter Laplacian")
                P_target = round((E + 1) / (S + len(self.clss_lst)), 4)
            else :
                P_target = round(E / S, 4)
            self.prob_dict[f"prob_{values}_{_}"] = P_target
            print(f"prob_{values}_{_}", P_target)
        laplacian_flag = False
            
    def get_guss_prob(self, feature, values):
        
        for _ in self.clss_lst:
            guss_list = df[df[self.target] == _][feature].values
            mean = guss_list.mean().round(4)
            sigma = guss_list.std().round(4)
            self.prob_dict[f"std_{feature}_{_}"] = sigma
            self.prob_dict[f"mean_{feature}_{_}"] = mean
            
            a = (1 / (sigma * np.sqrt(2*np.pi)))
            b = np.exp(-0.5*((values-mean)/sigma)**2)
            
            # print(mean, sigma, a, b)
            self.prob_dict[f"prob_{feature}_{_}"] = (a * b).round(4)
            print(f"prob_{feature}_{_}", self.prob_dict[f"prob_{feature}_{_}"])
            
    def get_bays_value(self, input_dict):
        
        for key, values in input_dict.items():
            if df[key].dtype != 'O':
                self.get_guss_prob(key, values)
            else :
                self.get_cond_prob(key, values)
                    
        for _ in self.clss_lst:
            bays_value = self.prob_dict[f"prob_{_}"]
            for key, values in input_dict.items():
                if df[key].dtype != 'O':
                    # print(f"prob_{key}_{_}", self.prob_dict[f"prob_{key}_{_}"])
                    bays_value = bays_value * self.prob_dict[f"prob_{key}_{_}"]
                else :
                    # print(f"prob_{values}_{_}", self.prob_dict[f"prob_{values}_{_}"])
                    bays_value = bays_value * self.prob_dict[f"prob_{values}_{_}"]
            self.prob_dict[f"bays_value_{_}"] = bays_value.round(4)
            
    def predict(self, input_dict):
        self.get_bays_value(input_dict)
        maximum = 0
        for _ in self.clss_lst:
            print(f"{_}", self.prob_dict[f"bays_value_{_}"])
            if self.prob_dict[f"bays_value_{_}"] > maximum:
                maximum = self.prob_dict[f"bays_value_{_}"]
                ans = _
        return ans

In [7]:
prob_df = naiveBayes(df, 'Stolen')
# prob_df.prob_dict

In [8]:
input_dict = {'Color': 'Red', 'Type': 'SUV',  'Mileage':12.6, 'Origin':'Imported'}
prob_df.predict(input_dict)

prob_Red_Yes 0.5
prob_Red_No 0.3333
prob_SUV_Yes 0.5
prob_SUV_No 1.0
prob_Mileage_Yes 0.79
prob_Mileage_No 0.2121
prob_Imported_Yes 0.5
prob_Imported_No 0.6667
Yes 0.0564
No 0.0202


'Yes'

In [9]:
prob_df.prob_dict

{'total': 7,
 'total_Yes': 4,
 'prob_Yes': 0.5714,
 'total_No': 3,
 'prob_No': 0.4286,
 'total_Red_Yes': 2,
 'prob_Red_Yes': 0.5,
 'total_Red_No': 1,
 'prob_Red_No': 0.3333,
 'total_SUV_Yes': 2,
 'prob_SUV_Yes': 0.5,
 'total_SUV_No': 3,
 'prob_SUV_No': 1.0,
 'std_Mileage_Yes': 0.5025,
 'mean_Mileage_Yes': 12.65,
 'prob_Mileage_Yes': 0.79,
 'std_Mileage_No': 1.3491,
 'mean_Mileage_No': 13.7,
 'prob_Mileage_No': 0.2121,
 'total_Imported_Yes': 2,
 'prob_Imported_Yes': 0.5,
 'total_Imported_No': 2,
 'prob_Imported_No': 0.6667,
 'bays_value_Yes': 0.0564,
 'bays_value_No': 0.0202}

In [10]:
input_dict = {'Color': 'Yellow', 'Type': 'SUV',  'Mileage':13.2, 'Origin':'Domestic'}
prob_df.predict(input_dict)

prob_Yellow_Yes 0.5
Encounter Laplacian
prob_Yellow_No 0.2
prob_SUV_Yes 0.5
prob_SUV_No 1.0
prob_Mileage_Yes 0.4361
prob_Mileage_No 0.2761
prob_Domestic_Yes 0.5
prob_Domestic_No 0.3333
Yes 0.0311
No 0.0079


'Yes'

In [11]:
prob_df.prob_dict

{'total': 7,
 'total_Yes': 4,
 'prob_Yes': 0.5714,
 'total_No': 3,
 'prob_No': 0.4286,
 'total_Red_Yes': 2,
 'prob_Red_Yes': 0.5,
 'total_Red_No': 1,
 'prob_Red_No': 0.3333,
 'total_SUV_Yes': 2,
 'prob_SUV_Yes': 0.5,
 'total_SUV_No': 3,
 'prob_SUV_No': 1.0,
 'std_Mileage_Yes': 0.5025,
 'mean_Mileage_Yes': 12.65,
 'prob_Mileage_Yes': 0.4361,
 'std_Mileage_No': 1.3491,
 'mean_Mileage_No': 13.7,
 'prob_Mileage_No': 0.2761,
 'total_Imported_Yes': 2,
 'prob_Imported_Yes': 0.5,
 'total_Imported_No': 2,
 'prob_Imported_No': 0.6667,
 'bays_value_Yes': 0.0311,
 'bays_value_No': 0.0079,
 'total_Yellow_Yes': 2,
 'prob_Yellow_Yes': 0.5,
 'total_Yellow_No': 0,
 'prob_Yellow_No': 0.2,
 'total_Domestic_Yes': 2,
 'prob_Domestic_Yes': 0.5,
 'total_Domestic_No': 1,
 'prob_Domestic_No': 0.3333}

##### 2. Implement the Naïve Bayes classifier using inbuilt functions.

In [12]:
# load the iris dataset
df = pd.read_csv("iris_data.csv")

In [13]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [14]:
features = df.columns.tolist()
target = "class"
features.remove(target)
features, target

(['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'],
 'class')

In [15]:
# store the feature matrix (X) and response vector (y)
X = np.array(df[features])
y = np.array(df[target])

In [16]:
# splitting X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [17]:
# training the model on training set
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [18]:
# making predictions on the testing set
y_pred = gnb.predict(X_test)

In [19]:
# comparing actual response values (y_test) with predicted response values (y_pred)
from sklearn import metrics
print("Gaussian Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)

Gaussian Naive Bayes model accuracy(in %): 97.36842105263158


In [20]:
# To predict the Final Result.
gnb.predict([[5.1, 3.5, 1.4, 0.2]])

array(['setosa'], dtype='<U10')